In [1]:
import json
import statistics
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

In [19]:
def load_json(path):
    with open('../data/moby/jsons/' + path) as f:
        return json.load(f)

response_to_item = load_json('child_response_to_item.json')
item_to_form = load_json('item_to_form.json')
form_to_grade = load_json('form_to_grade.json')

session_to_annote_category_to_scores = load_json('session_to_annote_category_to_scores.json')
response_to_annote_category_to_score = load_json('response_to_annote_category_to_scores.json')

response_to_session = load_json('child_response_to_session.json')

# child_response_to_words = load_json('child_response_to_words.json')
# child_session_to_responses = load_json('child_session_to_responses.json')

# gold_response_to_words = load_json('gold_response_to_words.json')
# gold_session_to_responses = load_json('gold_session_to_responses.json')

In [22]:
col_response = []
col_session = []
col_perfect = []
col_not_too_slow = []
col_not_too_fast = []
col_item = []
col_form = []
col_grade = []

for response, annotate_category_to_score in response_to_annote_category_to_score.items():
    if '11' not in annotate_category_to_score:
        continue
    
    annotes = annotate_category_to_score['11']
    
    perfect = 1 if all(annote == 2 for annote in annotes) else 0
    not_too_slow = 1 if 1 not in annotes else 0
    not_too_fast = 1 if 3 not in annotes else 0
    
    
    
    if response in response_to_item and response in response_to_session:
        session = response_to_session[response]
        item = response_to_item[response]
        form = item_to_form[str(item)]
        grade = form_to_grade[form]
        
        col_response.append(response)
        col_session.append(session)
        col_perfect.append(perfect)
        col_not_too_slow.append(not_too_slow)
        col_not_too_fast.append(not_too_fast)
        col_item.append(item)
        col_form.append(form)
        col_grade.append(grade)

In [24]:
df = pd.DataFrame({
    'response': col_response,
    'session': col_session,
    'is_perfect': col_perfect,
    'not_too_slow': col_not_too_slow,
    'not_too_fast': col_not_too_fast,
    'item': col_item,
    'form': col_form,
    'grade': col_grade
})

In [36]:
orig_matrix = pd.read_csv('../data/moby/annotes.csv')

In [37]:
orig_matrix

,Stack,Session,WCPM,Acc,Comp,Exp,level,mlevel,Form,Response,...,11,12,13,14,15,16,17,18,19,20
0,362,41664.0,36.0,0.847,3.4,1.4,6.0,mF,Grade2Spring,693669.0,...,1.0,2.0,2.0,1.0,2.0,2.0,2.0,NaN,2.0,1.0
1,362,47152.0,155.0,1.000,6.5,4.0,18.0,mQ,Grade2Spring,748431.0,...,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,1.0
2,362,32210.0,41.0,0.776,5.1,1.8,6.0,mF,Grade2Spring,565127.0,...,1.0,1.0,2.0,2.0,3.0,1.0,1.0,2.0,1.0,1.0
3,362,48359.0,134.0,0.992,5.0,4.0,16.0,mP,Grade2Spring,760144.0,...,3.0,3.0,3.0,3.0,1.0,2.0,1.0,3.0,2.0,1.0
4,362,52812.0,50.0,0.908,2.4,1.9,7.0,mG,Grade2Spring,805147.0,...,2.0,2.0,2.0,2.0,NaN,1.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7833,370,28135.0,83.0,0.971,5.8,2.6,12.0,mL,Grade4Fall,510072.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
7834,370,30306.0,32.0,0.667,2.7,2.0,7.0,mJ,Grade4Fall,536550.0,...,2.0,3.0,3.0,3.0,2.0,3.0,3.0,2.0,2.0,3.0
7835,370,30306.0,32.0,0.667,2.7,2.0,7.0,mJ,Grade4Fall,536542.0,...,2.0,3.0,3.0,3.0,1.0,3.0,3.0,2.0,2.0,3.0
7836,370,34943.0,49.0,0.990,2.1,2.1,9.0,mJ,Grade4Fall,609163.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
response_to_session_wcpm = {}
response_to_session_acc = {}
response_to_response_wcpm = {}
for idx, row in orig_matrix.iterrows():
    if row.Response != row.Response:
        continue
    response = str(int(row.Response))
    response_to_session_wcpm[response] = row.WCPM
    response_to_session_acc[response] = row.Acc
    response_to_response_wcpm[response] = row.WCPM_Response

In [47]:
df['session_wcpm'] = [response_to_session_wcpm[r] for r in df.response]
df['session_acc'] = [response_to_session_acc[r] for r in df.response]
df['response_wcpm'] = [response_to_response_wcpm[r] for r in df.response]

In [50]:
df.to_csv('../reading-rate-prediction/better_table_for_predicting_rate.csv', index=None)